In [1]:
import pandas as pd
import numpy as np


In [2]:
from google.colab import drive
drive.mount('/content/gdrive') # source : https://developer.imdb.com/non-commercial-datasets/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
df_title_basics = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/IMDB_DATASETS/data_title_basics.tsv", sep='\t', dtype={'isAdult': 'object'})
print('title_basics length: ','\t',len(df_title_basics))

title_basics length:  	 10082801


In [4]:
value_to_replace_1 = r'\\N'
value_to_replace_2 = r'\N'

df_title_basics.replace(value_to_replace_1, np.nan, inplace=True)
df_title_basics.replace(value_to_replace_2, np.nan, inplace=True)


# **Cleaning**

---



## 1. Title_basics table


In [5]:
print('rows in dataframe: ',len(df_title_basics))
print('Nulls per column')
print(df_title_basics.isnull().sum())
df_title_basics.head(2)

rows in dataframe:  10082801
Nulls per column
tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1353425
endYear           9971602
runtimeMinutes    7082117
genres             453418
dtype: int64


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"


### 1.1 isAdult column

In [6]:
print(len(df_title_basics))
df_title_basics['isAdult'].unique()

10082801


array(['0', '1', '2019', '1981', '2020', '2017', nan, '2023', '2022',
       '2011', '2014', '2005'], dtype=object)

In [7]:
wrong_values_years = [2019,1981,2020,2017,2023,2022,2011,2014,2005]
string_values = ['0','1']
print('Number of wrong values:','\t',len(df_title_basics[~df_title_basics['isAdult'].isin(string_values)]))
df_title_basics = df_title_basics[df_title_basics['isAdult'].isin(string_values)]
# keep only values intended for the column, they're still a string type -> change to int
print('number of correct values:','\t',len(df_title_basics))
df_title_basics['isAdult'] = df_title_basics['isAdult'].astype(str).astype(int)
df_title_basics['isAdult'].unique()

Number of wrong values: 	 17
number of correct values: 	 10082784


array([0, 1])

#### Wrong value rows are deleted

---






### 1.2 tconst column


In [8]:
pattern = r'^tt\d{7,8}$'   ## code rule for column must be followed on every value
invalid_values = df_title_basics[~df_title_basics['tconst'].str.match(pattern)]  #check if rule is followed
print("Invalid values:",'\t',len(invalid_values))
print('total rows: ','\t\t',len(df_title_basics))
print('unique values: ','\t',len(df_title_basics['tconst'].unique()))
# clean

Invalid values: 	 0
total rows:  		 10082784
unique values:  	 10082784


### 1.3 titleType column

In [9]:
contains_non_string = df_title_basics['titleType'][df_title_basics['titleType'].apply(type) != str].any()
print(contains_non_string)
df_title_basics['titleType'].unique()
# clean

False


array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

### 1.4 primaryTitle column

In [10]:
print(df_title_basics['primaryTitle'].info())
print('Number of nan values','\t',len(df_title_basics[df_title_basics['primaryTitle'].isna()]))
# the nan values in primaryTitle column are the same as originalTitle column

<class 'pandas.core.series.Series'>
Int64Index: 10082784 entries, 0 to 10082800
Series name: primaryTitle
Non-Null Count     Dtype 
--------------     ----- 
10082773 non-null  object
dtypes: object(1)
memory usage: 153.9+ MB
None
Number of nan values 	 11


In [11]:
df_title_basics.dropna(subset='primaryTitle',inplace=True)

### 1.5 startYear & endYear column

In [12]:
df_title_basics['startYear'] = df_title_basics['startYear'].astype('Int64')
df_title_basics['endYear'] = df_title_basics['endYear'].astype('Int64')
df_title_basics['runtimeMinutes'] = df_title_basics['runtimeMinutes'].astype('Int64')


### 1.6 Genres column (explode)

In [13]:
print('unique values: \t',len(df_title_basics['genres'].unique()))
df_title_basics['genres'] = df_title_basics['genres'].str.split(',')
df_title_basics = df_title_basics.explode('genres').reset_index(drop=True)
print('actual unique values: \t',len(df_title_basics['genres'].unique()))

unique values: 	 2342
actual unique values: 	 29


In [14]:
print('n° of rows: ',len(df_title_basics))
df_title_basics = df_title_basics[(df_title_basics['titleType']!='tvEpisode')]
print('highest start Year: ',df_title_basics['startYear'].max())                      # returns 2031 -> deleting all values over 2023
print('lowest start Year: ',df_title_basics['startYear'].min())
df_title_basics = df_title_basics[df_title_basics['startYear']<=2023]
df_title_basics = df_title_basics[df_title_basics['endYear']<=2023]
print('new n° of rows: ',len(df_title_basics))

n° of rows:  16077020
highest start Year:  2031
lowest start Year:  1874
new n° of rows:  163462


#### Starting years have values higher than the present year.


#### Values over 2023 dropped


---




In [15]:
df_title_basics.describe()

,isAdult,startYear,endYear,runtimeMinutes
count,163462.000000,163462.0,163462.0,87596.0
mean,0.005677,2004.164632,2005.887228,58.620713
std,0.075133,17.217478,16.477579,101.474531
min,0.000000,1900.0,1906.0,1.0
25%,0.000000,1996.0,1998.0,25.0
50%,0.000000,2010.0,2012.0,37.0
75%,0.000000,2017.0,2018.0,60.0
max,1.000000,2023.0,2023.0,8400.0


In [16]:
df_title_basics.drop(columns=['isAdult','startYear','endYear','runtimeMinutes']).describe()

,tconst,titleType,primaryTitle,originalTitle,genres
count,163462,163462,163462,163462,157294
unique,111081,2,105571,106051,27
top,tt0262168,tvSeries,Shock Theatre,Shock Theatre,Comedy
freq,3,127375,84,87,26248


In [17]:
df_title_basics.to_csv('/content/gdrive/MyDrive/Colab Notebooks/IMDB_DATASETS/data_title_basics_c.tsv',sep='\t', index=False)   # save

## Ratings table

In [18]:
df_ratings = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/IMDB_DATASETS/data_ratings.tsv", sep='\t')
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338119 entries, 0 to 1338118
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1338119 non-null  object 
 1   averageRating  1338119 non-null  float64
 2   numVotes       1338119 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.6+ MB


In [19]:
print(len(df_ratings['tconst'].unique()))
print((df_ratings == value_to_replace_1).any().any())
print((df_ratings == value_to_replace_2).any().any())
df_ratings.to_csv('/content/gdrive/MyDrive/Colab Notebooks/IMDB_DATASETS/data_ratings_c.tsv',sep='\t', index=False)  # save
%reset -f

1338119
False
False


## Names table

In [20]:
import pandas as pd
import numpy as np


In [21]:
df_names = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/IMDB_DATASETS/data_name_basics.tsv", sep='\t')
print('names length: ','\t',len(df_names))
print((df_names == r'\\N').any().any()) #False
print((df_names == r'\N').any().any()) #True
df_names.replace(r'\N', np.nan, inplace=True)

names length:  	 12771013
False
True


In [22]:
df_names['birthYear'] = df_names['birthYear'].astype('float64')
df_names['deathYear'] = df_names['deathYear'].astype('float64')
df_names[df_names['birthYear']<100].sort_values(by='birthYear')

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
137410,nm0144482,Xavier Castano,1.0,NaN,"assistant_director,producer,director","tt1701210,tt2909116,tt0338512,tt0870921"
738866,nm0784172,Lucio Anneo Seneca,4.0,65.0,writer,"tt0049203,tt0218822,tt0237666,tt0972562"
8354667,nm4511421,Megan Liz Smith,12.0,NaN,"miscellaneous,actress","tt1966378,tt3416744,tt14300912,tt10183848"
5801257,nm1620433,Karen Abercrombie,13.0,NaN,"actress,producer,writer","tt7383268,tt6791350,tt8180386,tt3832914"
11797510,nm8810146,Paul Walsh,21.0,NaN,actor,"tt0098878,tt0314567,tt0083839,tt0093223"
407258,nm0430769,Flavius Josephus,37.0,95.0,"writer,miscellaneous","tt0049833,tt0273236,tt0237666,tt0009573"
3824086,nm1306202,Plutarch,46.0,122.0,writer,"tt0237666,tt0056937,tt0272979,tt0472473"
486828,nm0515385,Titus Livius,59.0,17.0,writer,tt0003740
11988576,nm9049180,Pliny the Younger,61.0,113.0,NaN,NaN
3828983,nm1306711,Suetonius,69.0,140.0,writer,"tt0092322,tt3395994,tt0237666,tt0056937"


### keeping only actors, actresses and directors

In [23]:
df_names = df_names[(df_names['primaryProfession']!='writer')|(df_names['birthYear']>1900)]
df_names.dropna(subset='knownForTitles',inplace=True)
print('dataframe length: ',len(df_names))
df_names['knownForTitles'] = df_names['knownForTitles'].str.split(',')
df_names['primaryProfession'] = df_names['primaryProfession'].str.split(',')
df_names=df_names.explode('primaryProfession')
#print(df_names['primaryProfession'].unique())

primary_prof = ['actor','actress','director']
df_names = df_names[df_names['primaryProfession'].apply(lambda x: any(str(val) in str(x) for val in primary_prof))]

not_primary_prof = ['assistant_director','casting_director','art_director']
df_names = df_names[~df_names['primaryProfession'].apply(lambda x: any(str(val) in str(x) for val in not_primary_prof))]

print('dataframe length: ',len(df_names))
print('unique professions: ',df_names['primaryProfession'].unique())



dataframe length:  11237591
dataframe length:  5368886
unique professions:  ['actor' 'actress' 'director']


In [24]:
df_names_titles = df_names[['nconst','knownForTitles']].copy()
df_names.drop(columns='knownForTitles',inplace=True)
df_names_titles.drop_duplicates(subset='nconst',inplace=True)
df_names_professions = df_names[['nconst','primaryProfession']].copy()
df_names.drop(columns='primaryProfession',inplace=True)
df_names.drop_duplicates(subset='nconst',inplace=True)

In [25]:
df_names.to_csv('/content/gdrive/MyDrive/Colab Notebooks/IMDB_DATASETS/data_name_basics_c.tsv',sep='\t', index=False)
df_names.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5221517 entries, 0 to 12771003
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   nconst       object 
 1   primaryName  object 
 2   birthYear    float64
 3   deathYear    float64
dtypes: float64(2), object(2)
memory usage: 199.2+ MB


In [26]:
df_names_titles=df_names_titles.explode('knownForTitles').reset_index(drop=True)
df_names_titles.to_csv('/content/gdrive/MyDrive/Colab Notebooks/IMDB_DATASETS/data_name_titles_c.tsv',sep='\t', index=False)
df_names_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10273871 entries, 0 to 10273870
Data columns (total 2 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   nconst          object
 1   knownForTitles  object
dtypes: object(2)
memory usage: 156.8+ MB


In [27]:
df_names_professions.to_csv('/content/gdrive/MyDrive/Colab Notebooks/IMDB_DATASETS/data_name_professions_c.tsv',sep='\t', index=False)
df_names_professions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5368886 entries, 0 to 12771003
Data columns (total 2 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryProfession  object
dtypes: object(2)
memory usage: 122.9+ MB
